In [25]:
import numpy as np
import pickle
import requests

# API Keys
OPENWEATHER_API_KEY = "9a939355165997c5bc96fe69ce3370cd"
WAQI_API_KEY = "99e2255e97e295eea3d1d01735b9faf64ed8a4e1"
NOMINATIM_API_URL = "https://nominatim.openstreetmap.org/search"

# Load the trained Random Forest model
with open(r'C:/Users/Siddhi Sankhe/Desktop/Aqi recommendation/train_model.pkl', 'rb') as file:
    rf_model = pickle.load(file)

# Function for rule-based advice based on AQI, weather conditions, etc.
def rule_based_logic(features):
    aqi = features[3]  # AQI
    temperature = features[0]  # Temperature
    humidity = features[1]  # Humidity
    visibility = features[2]  # Visibility

    # Rule-based advice for AQI
    if aqi > 150:
        pollution_advice = "High Pollution detected! It's recommended to avoid outdoor activities."
        health_warning = "Stay indoors or wear a mask if necessary."
    elif aqi > 100:
        pollution_advice = "Moderate pollution levels. Caution is advised if you are sensitive to air quality."
        health_warning = "Consider wearing a mask and avoiding strenuous outdoor activities."
    else:
        pollution_advice = "Air quality is good. Enjoy your outdoor activities safely."
        health_warning = "No specific precautions needed, but always stay hydrated."

    # Rule-based advice for weather conditions
    if temperature > 35:
        temperature_advice = "It's quite hot today! Stay hydrated and take breaks in the shade."
    elif humidity > 80:
        temperature_advice = "High humidity makes it feel even hotter. Take it easy and drink plenty of water."
    elif visibility < 500:
        temperature_advice = "Visibility is low today. Drive carefully or stay indoors if possible."
    else:
        temperature_advice = "Weather looks fine. Proceed as usual."

    # Combine pollution and weather advice
    combined_advice = f"{pollution_advice} {health_warning} {temperature_advice}"

    return combined_advice

# Function for the hybrid model that combines rule-based logic and RF prediction
def hybrid_model(features):
    # Apply rule-based logic
    rb_output = rule_based_logic(features)

    # Use all features (adjust this based on how your model was trained)
    rf_input = np.array(features[:6]).reshape(1, -1)  # Assuming you use 6 features
    rf_output = rf_model.predict(rf_input)

    # Interpret the prediction
    if rf_output == 0:
        rf_interpretation = "Air quality is expected to be good."
    elif rf_output == 1:
        rf_interpretation = "Air quality is moderate. Proceed with caution."
    else:
        rf_interpretation = "Air quality is poor. Take necessary precautions."

    # Return both rule-based and RF model outputs
    return rb_output, rf_interpretation

# Fetch detailed weather data from OpenWeather API
def fetch_weather_data(lat, lon):
    url = f'http://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={OPENWEATHER_API_KEY}'
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        temperature = data['main']['temp'] - 273.15  # Convert from Kelvin to Celsius
        weather_description = data['weather'][0]['description']
        return f"Weather: {weather_description}, Temp: {temperature:.2f}°C"
    else:
        return "Error fetching weather data."

# Fetch pollution statistics from WAQI API
def fetch_pollution_data(city):
    url = f'http://api.waqi.info/feed/{city}/?token={WAQI_API_KEY}'
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        aqi = data['data']['aqi']
        dominant_pollutant = data['data']['dominentpol']
        return f"AQI: {aqi}, Dominant Pollutant: {dominant_pollutant}"
    else:
        return "Error fetching pollution data."

# Fetch safer routes using OSRM API (Open Street Map Routing)
def fetch_safer_routes(start_lat, start_lon, end_lat, end_lon):
    # Construct the OSRM API request URL for routing
    url = f'http://router.project-osrm.org/route/v1/driving/{start_lon},{start_lat};{end_lon},{end_lat}?overview=false'
    
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        routes = data['routes']
        
        if routes:
            route_info = routes[0]
            distance = route_info['legs'][0]['distance'] / 1000  # in kilometers
            duration = route_info['legs'][0]['duration'] / 60  # in minutes
            return f"Safer route: Distance: {distance:.2f} km, Duration: {duration:.2f} minutes"
        else:
            return "No safe routes found."
    else:
        return "Error fetching routes data."

# Function to get latitude and longitude from place name or pincode
def get_lat_lon_from_address(address):
    params = {
        'q': address,   # Place name or pincode
        'format': 'json'
    }
    response = requests.get(NOMINATIM_API_URL, params=params)
    
    if response.status_code == 200:
        data = response.json()
        if data:
            lat = data[0]['lat']
            lon = data[0]['lon']
            return float(lat), float(lon)
        else:
            return None, None
    else:
        return None, None

# Function to format the chatbot response with suggestions
def format_chatbot_response(rb_output, rf_output):
    return {
        "response": f"{rb_output} \n\nPrediction: {rf_output}",
        "suggestions": [
            "Would you like to get detailed weather data?",
            "Would you like to see the latest pollution statistics?",
            "Would you like to know safer routes to travel?"
        ]
    }

# Handle user's choice for next actions
def handle_user_choice(choice, lat, lon, city, end_lat, end_lon):
    if choice == 1:
        # Fetch detailed weather data
        return fetch_weather_data(lat, lon)
    elif choice == 2:
        # Fetch pollution data
        return fetch_pollution_data(city)
    elif choice == 3:
        # Fetch safer routes using OpenStreetMap (OSRM)
        return fetch_safer_routes(lat, lon, end_lat, end_lon)
    else:
        return "Invalid choice. Please select a valid option."


# Function to simulate chatbot flow
# Function to simulate chatbot flow
def simulate_chatbot_flow():
    # Example rule-based output and Random Forest prediction output
    rb_output = "High Pollution detected! It's recommended to avoid outdoor activities. Stay indoors or wear a mask if necessary."
    rf_output = "Air quality is poor. Take necessary precautions."

    # Get initial chatbot response with suggestions
    initial_response = format_chatbot_response(rb_output, rf_output)
    print(initial_response["response"])

    # Simulate user input (for example, they want to know the safer routes)
    user_choice = int(input("Enter the number of your choice: "))  # User input simulation

    # Coordinates for Mumbai (or any city) and pollution data city name
    lat, lon = 19.0760, 72.8777  # Example coordinates for Mumbai
    city = "mumbai"  # City for pollution data

    # If the user selects option 2 (pollution data), fetch pollution data without asking for destination
    if user_choice == 2:
        print(fetch_pollution_data(city))  # Directly fetch pollution data
        return  # Exit after providing the pollution data

    # For weather or safer routes (choices 1 or 3), ask for destination
    destination = input("Enter your destination (place name or pincode): ")
    end_lat, end_lon = get_lat_lon_from_address(destination)

    if end_lat and end_lon:
        # Handle the user's choice and provide the appropriate response
        if user_choice == 1:
            # Fetch weather data
            print(fetch_weather_data(lat, lon))
        elif user_choice == 3:
            # Fetch safer routes using OpenStreetMap (OSRM)
            print(fetch_safer_routes(lat, lon, end_lat, end_lon))
        else:
            print("Invalid choice. Please select a valid option.")
    else:
        print("Unable to find the destination. Please check your input.")

# Run the chatbot simulation
simulate_chatbot_flow()


High Pollution detected! It's recommended to avoid outdoor activities. Stay indoors or wear a mask if necessary. 

Prediction: Air quality is poor. Take necessary precautions.
Unable to find the destination. Please check your input.


In [36]:
import numpy as np
import pickle
import requests
from geopy.geocoders import Nominatim

# Load the trained Random Forest model
with open(r'C:/Users/Siddhi Sankhe/Desktop/Aqi recommendation/train_model.pkl', 'rb') as file:
    rf_model = pickle.load(file)

# API Keys (Please replace these with your actual keys)
WAQI_API_KEY = '99e2255e97e295eea3d1d01735b9faf64ed8a4e1'
OPENWEATHER_API_KEY = '9a939355165997c5bc96fe69ce3370cd'

# Rule-based logic for AQI insights
def rule_based_logic(aqi):
    if aqi > 150:
        return "High Pollution detected! It's recommended to avoid outdoor activities. Please wear masks while going outside."
    elif aqi > 100:
        return "Moderate pollution levels. Caution is advised if you are sensitive to air quality."
    else:
        return "Air quality is good. Enjoy your outdoor activities safely."

# Function to fetch pollution data from WAQI API
def fetch_pollution_data(city):
    url = f'http://api.waqi.info/feed/{city}/?token=99e2255e97e295eea3d1d01735b9faf64ed8a4e1'
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        aqi = data['data']['aqi']
        return aqi
    else:
        return None

# Function to fetch weather data from OpenWeather API
def fetch_weather_data(lat, lon):
    url = f'http://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid=9a939355165997c5bc96fe69ce3370cd'
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        temperature = data['main']['temp'] - 273.15  # Convert from Kelvin to Celsius
        weather_description = data['weather'][0]['description']
        return f"Weather: {weather_description}, Temp: {temperature:.2f}°C"
    else:
        return "Error fetching weather data."
    
geolocator = Nominatim(user_agent="my_geopy_app")

# Function to get latitude and longitude from a place name
def get_coordinates(place_name):
    location = geolocator.geocode(place_name)
    if location:
        return (location.latitude, location.longitude)
    else:
        return None


# Function to fetch traffic data from OpenStreetMap API
def get_traffic_data(start_location, end_location):
    url = f"https://router.project-osrm.org/route/v1/driving/{start_location[1]},{start_location[0]};{end_location[1]},{end_location[0]}?overview=full"
    response = requests.get(url)
    return response.json()

# Function to suggest safer routes
def suggest_safer_routes(start_place, end_place):
    start_coordinates = get_coordinates(start_place)
    end_coordinates = get_coordinates(end_place)

    if not start_coordinates or not end_coordinates:
        return "Could not find coordinates for one of the locations."

    traffic_data = get_traffic_data(start_coordinates, end_coordinates)

    if traffic_data and 'routes' in traffic_data:
        routes = traffic_data['routes']
        recommendations = []
        for route in routes:
            distance = route['distance'] / 1000  # Convert to kilometers
            duration = route['duration'] / 60  # Convert to minutes
            recommendations.append(f"Route: {distance:.2f} km, Estimated time: {duration:.2f} minutes")
        return recommendations
    else:
        return ["No routes found."]

# Function to simulate the chatbot's response and choices
def chatbot_response():
    city = "Mumbai"
    aqi = fetch_pollution_data(city)
    
    if aqi is None:
        return "Error fetching pollution data."
    
    # Get insights based on AQI
    insights = rule_based_logic(aqi)
    print(f"Today's AQI in {city}: {aqi}. {insights}")

    # User choices
    print("\nWhat would you like to do?")
    print("1. Get detailed weather data")
    print("2. See the latest pollution statistics")
    print("3. Know safer routes to travel")
    
    user_choice = input("Please enter the number of your choice: ")
    
    if user_choice == '1':
        lat, lon = 19.0760, 72.8777  # Mumbai coordinates
        weather_info = fetch_weather_data(lat, lon)
        print(weather_info)
        
    elif user_choice == '2':
        print(f"Latest pollution statistics for {city}: AQI: {aqi}")
        
    elif user_choice == '3':
        print("Welcome to the Travel Assistant!")
        start_place = input("Please enter your starting location: ")
        end_place = input("Please enter your destination: ")

        safer_routes = suggest_safer_routes(start_place, end_place)
        print("Safer route recommendations:")
        for route in safer_routes:
            print(route)
    else:
        print("Invalid choice. Please select a valid option.")

if __name__ == "__main__":
    chatbot_response()

Today's AQI in Mumbai: 177. High Pollution detected! It's recommended to avoid outdoor activities. Please wear masks while going outside.

What would you like to do?
1. Get detailed weather data
2. See the latest pollution statistics
3. Know safer routes to travel
Welcome to the Travel Assistant!
Safer route recommendations:
Route: 151.52 km, Estimated time: 119.16 minutes


In [32]:
pip install geopy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [33]:
import requests
from geopy.geocoders import Nominatim

# Initialize Nominatim API with a user agent
geolocator = Nominatim(user_agent="my_geopy_app")

# Function to get latitude and longitude from a place name
def get_coordinates(place_name):
    location = geolocator.geocode(place_name)
    if location:
        return (location.latitude, location.longitude)
    else:
        return None

# Function to fetch traffic data from OpenStreetMap API
def get_traffic_data(start_location, end_location):
    url = f"https://router.project-osrm.org/route/v1/driving/{start_location[1]},{start_location[0]};{end_location[1]},{end_location[0]}?overview=full"
    response = requests.get(url)
    return response.json()

# Function to suggest safer routes
def suggest_safer_routes(start_place, end_place):
    start_coordinates = get_coordinates(start_place)
    end_coordinates = get_coordinates(end_place)

    if not start_coordinates or not end_coordinates:
        return "Could not find coordinates for one of the locations."

    traffic_data = get_traffic_data(start_coordinates, end_coordinates)

    if traffic_data and 'routes' in traffic_data:
        routes = traffic_data['routes']
        recommendations = []
        for route in routes:
            distance = route['distance'] / 1000  # Convert to kilometers
            duration = route['duration'] / 60  # Convert to minutes
            recommendations.append(f"Route: {distance:.2f} km, Estimated time: {duration:.2f} minutes")
        return recommendations
    else:
        return ["No routes found."]

# Main function to run the chatbot
def chatbot_response():
    print("Welcome to the Travel Assistant!")
    start_place = input("Please enter your starting location: ")
    end_place = input("Please enter your destination: ")

    safer_routes = suggest_safer_routes(start_place, end_place)
    print("Safer route recommendations:")
    for route in safer_routes:
        print(route)

# Run the chatbot
if __name__ == "__main__":
    chatbot_response()

Welcome to the Travel Assistant!
Safer route recommendations:
Route: 149.04 km, Estimated time: 116.06 minutes
